In [1]:
# !pip install koco
import koco

train_dev = koco.load_dataset("korean-hate-speech", mode="train_dev")
unlabeled = koco.load_dataset("korean-hate-speech", mode="unlabeled")
test = koco.load_dataset("korean-hate-speech", mode="test")

curse = []
with open("./curse_detection.txt", "r") as f:
    for line in f.readlines():
        curse.append(line.strip())

In [2]:
from pprint import pprint

print("="*20, "train_dev", "="*25)
print()
print("type of 'train_dev' dataset:", type(train_dev))
print("keys in 'train_dev' dataset:", train_dev.keys())
print("total length of 'train_dev' dataset:", len(train_dev['train'])+len(train_dev['dev']))
print("sample data from 'train_dev' dataset")
pprint(train_dev['train'][0])
print()
print("="*20, "unlabeled", "="*25)
print()
print("type of 'unlabeled' dataset:", type(unlabeled))
print("total length of 'unlabeled' dataset:", len(unlabeled))
print("sample data from 'unlabeled' dataset")
pprint(unlabeled[0])
print()
print("="*22, "test", "="*25)
print()
print("type of 'test' dataset:", type(test))
print("total length of 'test' dataset:", len(test))
print("sample data from 'unlabeled' dataset")
pprint(test[0])
print()
print("="*22, "curse", "="*25)
print()
print("type of 'curse' dataset:", type(curse))
print("total length of 'curse' dataset:", len(curse))
print("sample data from 'curse' dataset")
pprint(curse[0])

==================== train_dev =========================

type of 'train_dev' dataset: <class 'dict'>
keys in 'train_dev' dataset: dict_keys(['train', 'dev'])
total length of 'train_dev' dataset: 8367
sample data from 'train_dev' dataset
{'bias': 'others',
 'comments': '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 'contain_gender_bias': False,
 'hate': 'hate',
 'news_title': '"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"'}

==================== unlabeled =========================

type of 'unlabeled' dataset: <class 'list'>
total length of 'unlabeled' dataset: 2033893
sample data from 'unlabeled' dataset
{'comments': '지드래곤은 난봉꾼이란...댓글도 달렸네 ㅋㅋ 이주연 학창시절 사진 보고 와라. 요즘 웬만한 여자 연예인하고 '
             '붙여놔도....미모가 최고였단다.ㅋ 5대 얼짱 출신.',
 'news_title': '"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"'}

====================== test =========================

type of 'test' dataset: <class 'list'>
total length of 'test' dataset: 974
sample data from 'unlabeled' dataset
{'comments': 'ㅋㅋㅋㅋ 그래도 조아해주는 팬들

In [ ]:
import pandas as pd

'''
텍스트 길이가 128이 넘어가면 커트
(트위치 도네 글자수 제한 기본이 128자)
출처: https://tgd.kr/g/tip/41829845
'''
hate_list = []
for txt in train_dev['train']:
    if len(txt['comments']) > 128:
        continue
    hate_ = 1 if txt['hate'] == 'hate' else 0
    curse_ = 0
    hate_list.append([txt['comments'], curse_, hate_, 0])

for txt in train_dev['dev']:
    if len(txt['comments']) > 128:
        continue
    hate_ = 1 if txt['hate'] == 'hate' else 0
    curse_ = 0
    hate_list.append([txt['comments'], curse_, hate_, 0])

for txt in test:
    if len(txt['comments']) > 128:
        continue
    hate_ = 0
    curse_ = 0
    hate_list.append([txt['comments'], curse_, hate_, 0])

for txt in unlabeled:
    if len(txt['comments']) > 128:
        continue
    hate_ = 0
    curse_ = 0
    hate_list.append([txt['comments'], curse_, hate_, 0])

df_hate = pd.DataFrame(hate_list[:], columns=['text', 'curse', 'hate', 'pass'])
print(len(df_hate))
df_hate.head()

In [3]:
# 복붙 도배 문장 걸러내는 함수
def principal_period(s):
    i = (s+s).find(s, 1, -1)
    return None if i == -1 else s[:i]

def check_repeat(user_chat):
    if principal_period(user_chat): # 복붙으로 도배한 문장 걸러서 통과 (띄어쓰기 없이 복붙)
        s = principal_period(user_chat)
        if len(s) <= 3: # 반복되는 문장이 3글자 이하면 1번 반복 시킴
            return s+s
        else: # 이외에는 반복 X
            return s
    
    elif principal_period(user_chat + " "): # 복붙으로 도배한 문장 걸러서 통과 (띄어쓰기 있는 복붙)
        s = principal_period(user_chat + " ")
        if len(s) <= 3:
            return s+s
        else:
            return s
        
    user_chat = user_chat[:-1] # 복붙 후 마지막에 v, ㅍ, ' 등이 붙는 경우
    
    # 위와 동일
    if principal_period(user_chat):
        s = principal_period(user_chat)
        if len(s) <= 3:
            return s+s
        else:
            return s
    
    elif principal_period(user_chat + " "):
        s = principal_period(user_chat + " ")
        if len(s) <= 3:
            return s+s
        else:
            return s
    
    return None # 해당 사항 없으면 None 리턴

In [4]:
import pandas as pd
from tqdm import tqdm
import re

'''
커스텀 데이터
'''
limit_len = 24 # 글자 수 제한
hate_list = []
for txt in train_dev['train']:
    if len(txt['comments']) > limit_len:
        continue
    hate_list.append(txt['comments'])

print("train_dev['train'] data finished")

for txt in train_dev['dev']:
    if len(txt['comments']) > limit_len:
        continue
    hate_list.append(txt['comments'])
    
print("train_dev['dev'] data finished")

for txt in test:
    if len(txt['comments']) > limit_len:
        continue
    hate_list.append(txt['comments'])
    
print("test data finished")

for txt in tqdm(unlabeled):
    if len(txt['comments']) > limit_len:
        continue
    hate_list.append(txt['comments'])

print("unlabeled data finished")

preprocessed_list = []

for t in tqdm(hate_list):

    user_chat = re.sub(r"\s+", r" ", t)
    user_chat = re.sub(r"[“”‘’]", r"\'", user_chat)
    user_chat = re.sub(r"[〈<＜「≪《『]", "<", user_chat)
    user_chat = re.sub(r"[〉>＞」≫》』]", ">", user_chat)
    user_chat = re.sub(r'[‥…]+', r'...', user_chat)
    user_chat = re.sub(r'[\?¿？]+', r'\?', user_chat)
    user_chat = re.sub(r'[!¡！]+', r'!', user_chat)
    user_chat = re.sub(r"([^\-—_=+,\./<>?\[\]{};:\'\"!@#$%\^&*\(\)₩`´~\|\\ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9ぁ-ゔゞァ-・ヽヾ゛゜ー一-龯\u3000-\u303F\u3400-\u4DBF\u4E00-\u9FFF\s]+)", r'', user_chat)
    
    if len(user_chat) == 0: continue
    
    elif user_chat[0] == '!': continue
    
    elif re.fullmatch(r"[ㄱㄲㅋㄷㄸㅎㅠㅜZEGzeg\-=\?!\d\s]+", user_chat): # z(ㅋ), e(ㄷ), ㅎ, G/g(gg) + 숫자만 있는 문장 제거
        continue
    
    elif re.fullmatch(r"[ぁ-ゔゞァ-・ヽヾ゛゜ー一-龯w\d\s]+", user_chat): # w, 숫자, 일본어만 있는 문장 제거
        continue
        
    elif re.fullmatch(r"[\u3000-\u303F\u3400-\u4DBF\u4E00-\u9FFF\d\s]+", user_chat): # 영어, 숫자, 중국어만 있는 문장 제거
        continue
            
    elif re.fullmatch(r"[₩/\\\(\)\[\]<>\-_=+@#$%\^&*,\.;:\'\"\?!\s]+", user_chat): # 문장부호, 특수기호만 있는 문장 제거
        continue
    
    rep = check_repeat(user_chat)
    if rep: user_chat = rep
    
    rep_num = 0
    while re.search(r"(.)\1{4,}", fr"{user_chat}"):  # 4번 이상 반복되는 글자 3번만 반복되도록 수정
        rep_num += 1
        if rep_num > 20:
            print(user_chat)
            break
        repeated = re.search(r"(.)\1{4,}", fr"{user_chat}").group()
        if repeated[0] == '^':
            user_chat = re.sub(r'\^+', r'^^^', user_chat)
        try:
            user_chat = re.sub(repeated, repeated[0]*3, user_chat)
        except: break
            
    user_chat = re.sub(r"\\", "", user_chat) # 백슬래쉬 제거
    
    preprocessed_list.append(user_chat)
    
hate_list = list(set(preprocessed_list))
df_hate = pd.DataFrame(hate_list, columns=['text'])
print(len(df_hate))
df_hate.head(10)

train_dev['train'] data finished
train_dev['dev'] data finished
test data finished


100%|██████████| 2033893/2033893 [00:01<00:00, 1992446.09it/s]


unlabeled data finished


100%|██████████| 847871/847871 [00:22<00:00, 37945.00it/s]


733848


,text
0,하라언니 언제나 응원합니다
1,졷망따리 ㅋㅈㅋ
2,여신이따로없다 진심
3,"매드손, 마미크라운 완죤 떡관종이야 ㅋ"
4,한순이 어디있나요 ㅠㅠㅠ
5,넘나 이쁜 아가들.똘망똘망
6,어머 단발 너무 이쁘다
7,송중기생가 ㅋㅋ
8,나 나혼자산다 나올때부터 싫었다 음침하니
9,안 돌아와도 됩니다


In [5]:
beep_set = list(set(df_hate['text']))
print(len(beep_set))
pd.DataFrame(beep_set, columns=['text']).to_csv("./beepData.tsv", index=False, sep='\t')

733848


In [ ]:
import random

df = df_hate[2 < df_hate.length]
idx = random.sample(range(len(df)), 500)
df = df.iloc[idx]
df['text'].to_csv("./dev.csv", index=False)

In [ ]:
curse_list = []
for txt in curse:
    tmp = txt.split('|')
    if len(tmp) > 2: # 중간에 '|' 기호 들어가는 문장 스킵
        continue
    elif len(tmp[0]) > 128: # 텍스트 길이 제한 128
        continue
    curse_list.append([tmp[0], tmp[1], 0, 0])

df_curse = pd.DataFrame(curse_list, columns=['text', 'curse', 'hate', 'pass'])
print(len(df_curse))
df_curse.head()

In [ ]:
df_hate.to_csv("/content/drive/MyDrive/nlp10-최종/hate.csv", index=False)
df_curse.to_csv("/content/drive/MyDrive/nlp10-최종/curse.csv", index=False)